In [11]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/3. Analyses/2. Data Processing/Output'
cur_date = '012425'

library(missForest)
library(readxl)
library(tidyverse)
library(imputeLCMD)
library(factoextra)
# library(vegan)
# library(preprocessCore)

# reading in files
ws_df = data.frame(Data = 'WS', read_excel("Input/Woodsmoke_Data_012325.xlsx", sheet = 2))
wf_df = data.frame(Data = 'WF', read_excel("Input/Wildfire_Data_012325.xlsx", sheet = 2)) %>%
    # removing samples that weren't measured at all 
    filter(!is.na(Replicate))

In [12]:
head(ws_df)
head(wf_df)

,Data,HAWC_ID,Study,Replicate,Sample_Number,Chemical_Class,Metric,DTXSID,Name,Sample_Identifier,Value
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,429174,Arif et al. 2017,1,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,DTXSID1074759_PAH_1,NA
2,WS,429174,Arif et al. 2017,2,2,PAH,Weight,DTXSID1074759,1-Methylchrysene,DTXSID1074759_PAH_2,NA
3,WS,429174,Arif et al. 2017,1,3,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,DTXSID9020877_PAH_3,NA
4,WS,429174,Arif et al. 2017,2,4,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,DTXSID9020877_PAH_4,NA
5,WS,429174,Arif et al. 2017,1,5,PAH,Weight,DTXSID9020877,1-Methylphenanthrene,DTXSID9020877_PAH_5,NA
6,WS,429174,Arif et al. 2017,2,6,PAH,Weight,DTXSID9020877,1-Methylphenanthrene,DTXSID9020877_PAH_6,NA


,Data,HAWC_ID,Study,Replicate,Sample_Number,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289821,Liang et al. 2021,1,40,PAH,Volume,NA,1-(10-methylanthracen-9-yl)ethanone,5
2,WF,1289821,Liang et al. 2021,1,40,PAH,Volume,DTXSID50176885,1-Acenaphthenone,1
3,WF,1289895,Silberstein et al. 2023,1,34,PAH,Volume,DTXSID1074759,1-Methylchrysene,0.01
4,WF,1289777,Harper et al. 2019,1,12,PAH,Weight,DTXSID0025654,1-Methylpyrene,9.7999999999999997E-3
5,WF,1289777,Harper et al. 2019,2,13,PAH,Weight,DTXSID0025654,1-Methylpyrene,1.4E-3
6,WF,1289777,Harper et al. 2019,3,14,PAH,Weight,DTXSID0025654,1-Methylpyrene,2.4500000000000001E-2


First need to save the non-detect data.

In [16]:
# creating a df for non-detect data that got lost when averaging
nd_ws_df = ws_df %>%
    filter(Value == 'ND') %>%
    select(-c("Sample_Number", "Sample_Identifier")) %>%
    unique()

nd_ws_df

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,1063040,Asita et al. 1991,1,PAH,Weight,DTXSID3052690,"Dibenzo(a,e)pyrene",ND
2,WS,271571,Dilger et al. 2016,1,PAH,Weight,DTXSID3021774,Acenaphthene,ND
3,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,ND
4,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,ND
5,WS,821855,Erlandsson et al. 2020,2,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,ND
6,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3021774,Acenaphthene,ND
7,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID8020913,Naphthalene,ND
8,WS,271638,Jalava et al. 2012,6,PAH,Weight,DTXSID6063143,5-Methylchrysene,ND
9,WS,271638,Jalava et al. 2012,4,PAH,Weight,DTXSID3021774,Acenaphthene,ND


Taking the average number of replicates.

In [8]:
ws_df$Value = as.numeric(ws_df$Value)
wf_df$Value = as.numeric(wf_df$Value)

avg_ws_df = ws_df %>%
    group_by(Study, Chemical_Class, Metric, Name) %>%
    mutate(Avg_Value = mean(Value)) %>% 
    select(-c("Replicate", "Sample_Number", "Value", "Sample_Identifier")) %>%
    unique() %>%
    ungroup()

avg_wf_df = wf_df %>%
    group_by(Study, Chemical_Class, Metric, Name) %>%
    mutate(Avg_Value = mean(Value)) %>% 
    select(-c("Replicate", "Sample_Number", "Value")) %>%
    unique() %>%
    ungroup()

avg_df = rbind(avg_ws_df, avg_wf_df)
head(avg_df)

Data,HAWC_ID,Study,Chemical_Class,Metric,DTXSID,Name,Avg_Value
<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
WS,429174,Arif et al. 2017,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
WS,429174,Arif et al. 2017,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,NA
WS,429174,Arif et al. 2017,PAH,Weight,DTXSID9020877,1-Methylphenanthrene,NA
WS,429174,Arif et al. 2017,PAH,Weight,DTXSID0025654,1-Methylpyrene,NA
WS,429174,Arif et al. 2017,PAH,Weight,DTXSID8060616,2-Methylanthracene,NA
WS,429174,Arif et al. 2017,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,NA


# Chemical/Metal Background Filter

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. Typically a variable is removed if it has less than 50% of woodsmoke (WS) and wildfire (WF) data.

In [ ]:
variable_presence_df = avg_df %>%
    mutate(count = ifelse(!is.na(Avg_Value), 1, 
                          ifelse(
                          0)) %>%
    group_by(Time_Point, Concentration, Condensate, Burn_Condition, mRNA) %>%
    # calculating the percentage of variables with data in all subjects
    summarize(Variable_Presence_Percentage = sum(count)/ length(unique(Subject_No)) * 100) %>%
    arrange(Variable_Presence_Percentage)

variable_presence_df %>%
    filter(Variable_Presence_Percentage < 75)